Продолжим работать по плану тестирования; на очереди — логика приложения:

1. Анонимный пользователь не может отправить комментарий.

2. Авторизованный пользователь может отправить комментарий.

3. Если комментарий содержит запрещённые слова, он не будет опубликован.

4. Авторизованный пользователь может редактировать или удалять свои комментарии.

5. Авторизованный пользователь не может редактировать или удалять чужие комментарии.

Первый пункт в плане может вызвать эффект deja vu: буквально в прошлом уроке уже был тест, проверяющий, что форма для комментирования недоступна анонимному пользователю. 

Но принцип независимости тестов требует проверить вариант, когда анонимный пользователь всё же смог отправить запрос на создание комментария. Нужно убедиться, что в такой ситуации комментарий не будет добавлен.

Исходя из той же логики следует убедиться, что авторизованный пользователь не может удалять или редактировать чужие комментарии. Мы уже проверяли, что страницы удаления и редактирования комментария недоступны посторонним по GET-запросу, но логика обработки разных запросов может отличаться; следовательно, нужно  проверить, что если POST- и DELETE-запросы отправлены не автором комментария — они будут отклонены.

Тесты стоит разместить в двух разных классах, ведь для них потребуются разные фикстуры:

1. Для тестирования отправки комментариев и запрета стоп-слов в фикстурах нужно создать новость и одного авторизованного пользователя (для неавторизованного применим клиент, который в тесте создаётся автоматически).

2. Для проверки доступа к редактированию и удалению комментариев понадобятся предварительно созданная новость, комментарий к ней и два авторизованных пользователя.

Импортируем в код всё необходимое для работы, создадим класс для первой группы тестов и подготовим в нём объекты, необходимые для тестирования. 


In [ ]:
# news/tests/test_logic.py
from http import HTTPStatus

from django.contrib.auth import get_user_model
from django.test import Client, TestCase
from django.urls import reverse

# Импортируем из файла с формами список стоп-слов и предупреждение формы.
# Загляните в news/forms.py, разберитесь с их назначением.
from news.forms import BAD_WORDS, WARNING
from news.models import Comment, News

User = get_user_model()


class TestCommentCreation(TestCase):
    # Текст комментария понадобится в нескольких местах кода, 
    # поэтому запишем его в атрибуты класса.
    COMMENT_TEXT = 'Текст комментария'

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(title='Заголовок', text='Текст')
        # Адрес страницы с новостью.
        cls.url = reverse('news:detail', args=(cls.news.id,))
        # Создаём пользователя и клиент, логинимся в клиенте.
        cls.user = User.objects.create(username='Мимо Крокодил')
        cls.auth_client = Client()
        cls.auth_client.force_login(cls.user)
        # Данные для POST-запроса при создании комментария.
        cls.form_data = {'text': cls.COMMENT_TEXT} 

***
## Проверка POST-запросов на добавление комментариев

Первый тест: убедимся, что анонимный пользователь не может создать комментарий. Для этого достаточно проверить, что при отправке POST-запроса на URL `/news/<pk>/` в системе не появилось новых комментариев. При запуске тестов создаётся пустая база данных, так что проверим, что после отправки запроса число объектов модели `Comment` останется равным нулю.


In [ ]:
# news/tests/test_logic.py
...
    def test_anonymous_user_cant_create_comment(self):
        # Совершаем запрос от анонимного клиента, в POST-запросе отправляем
        # предварительно подготовленные данные формы с текстом комментария.     
        self.client.post(self.url, data=self.form_data)
        # Считаем количество комментариев.
        comments_count = Comment.objects.count()
        # Ожидаем, что комментариев в базе нет - сравниваем с нулём.
        self.assertEqual(comments_count, 0)

Теперь проверим, что залогиненный пользователь может оставить комментарий: отправим POST-запрос через авторизованный клиент.

Затем посчитаем количество комментариев в системе: база была пустая, так что число комментариев должно стать равным единице. 

После этого проверим, что поля комментария содержат корректную информацию. В пустую тестовую базу был добавлен лишь один комментарий, так что получить его можно методом `objects.get()`, применять `.first()` или `.last()` необязательно.


In [ ]:
# news/tests/test_logic.py
...
    def test_user_can_create_comment(self):
        # Совершаем запрос через авторизованный клиент.
        response = self.auth_client.post(self.url, data=self.form_data)
        # Проверяем, что редирект привёл к разделу с комментами.
        self.assertRedirects(response, f'{self.url}#comments')
        # Считаем количество комментариев.
        comments_count = Comment.objects.count()
        # Убеждаемся, что есть один комментарий.
        self.assertEqual(comments_count, 1)
        # Получаем объект комментария из базы.
        comment = Comment.objects.get()
        # Проверяем, что все атрибуты комментария совпадают с ожидаемыми.
        self.assertEqual(comment.text, self.COMMENT_TEXT)
        self.assertEqual(comment.news, self.news)
        self.assertEqual(comment.author, self.user) 

После создания комментария пользователь перенаправляется на страницу отдельной записи в раздел с комментариями, поэтому к адресу страницы добавлено указание на нужный раздел: `#comments`. Такие указатели называют anchor, «якорь». Ссылка с якорем позволяет не просто сослаться на страницу, а прокрутить эту страницу до определённого раздела; в нашем случае — до HTML-тега с атрибутом `id="comments"`.

***
## Проверка блокировки стоп-слов

Теперь проверим, что пользователь не может использовать запрещённые слова при комментировании. Заодно проверим, что при обнаружении стоп-слов форма возвращает ошибку. Это можно сделать при помощи метода `assertFormError()`; в его аргументах указываются:

* объект `response`, в котором ожидается ошибка,

* `form` — имя формы (как оно указано в словаре `context`),

* `field` — имя поля формы, ошибку которого нужно протестировать,

* `errors` — ожидаемый текст ошибки или список ошибок.

In [ ]:

# news/tests/test_logic.py
...
    def test_user_cant_use_bad_words(self):
        # Формируем данные для отправки формы; текст включает
        # первое слово из списка стоп-слов.
        bad_words_data = {'text': f'Какой-то текст, {BAD_WORDS[0]}, еще текст'}
        # Отправляем запрос через авторизованный клиент.
        response = self.auth_client.post(self.url, data=bad_words_data)
        form = response.context['form']
        # Проверяем, есть ли в ответе ошибка формы.
        self.assertFormError(
            form=form,
            field='text',
            errors=WARNING
        )
        # Дополнительно убедимся, что комментарий не был создан.
        comments_count = Comment.objects.count()
        self.assertEqual(comments_count, 0) 


***
## Проверка удаления и редактирования комментария

Для оставшихся двух тестов создадим отдельный тестовый класс. На этапе подготовки данных создадим двух пользователей (автора комментария и читателя), новость и комментарий к ней, а также запишем в атрибуты класса адреса для редактирования и удаления комментария.


In [ ]:

# news/tests/test_logic.py
...
class TestCommentEditDelete(TestCase):
    # Тексты для комментариев не нужно дополнительно создавать 
    # (в отличие от объектов в БД), им не нужны ссылки на self или cls, 
    # поэтому их можно перечислить просто в атрибутах класса.
    COMMENT_TEXT = 'Текст комментария'
    NEW_COMMENT_TEXT = 'Обновлённый комментарий'

    @classmethod
    def setUpTestData(cls):
        # Создаём новость в БД.
        cls.news = News.objects.create(title='Заголовок', text='Текст')
        # Формируем адрес блока с комментариями, который понадобится для тестов.
        news_url = reverse('news:detail', args=(cls.news.id,))  # Адрес новости.
        cls.url_to_comments = news_url + '#comments'  # Адрес блока с комментариями.
        # Создаём пользователя - автора комментария.
        cls.author = User.objects.create(username='Автор комментария')
        # Создаём клиент для пользователя-автора.
        cls.author_client = Client()
        # "Логиним" пользователя в клиенте.
        cls.author_client.force_login(cls.author)
        # Делаем всё то же самое для пользователя-читателя.
        cls.reader = User.objects.create(username='Читатель')
        cls.reader_client = Client()
        cls.reader_client.force_login(cls.reader)
        # Создаём объект комментария.
        cls.comment = Comment.objects.create(
            news=cls.news,
            author=cls.author,
            text=cls.COMMENT_TEXT
        )
        # URL для редактирования комментария.
        cls.edit_url = reverse('news:edit', args=(cls.comment.id,)) 
        # URL для удаления комментария.
        cls.delete_url = reverse('news:delete', args=(cls.comment.id,))  
        # Формируем данные для POST-запроса по обновлению комментария.
        cls.form_data = {'text': cls.NEW_COMMENT_TEXT} 


Проверим, что автор может удалить свой комментарий:


In [ ]:

# news/tests/test_logic.py
...
    def test_author_can_delete_comment(self):
        # От имени автора комментария отправляем DELETE-запрос на удаление.
        response = self.author_client.delete(self.delete_url)
        # Проверяем, что редирект привёл к разделу с комментариями.
        self.assertRedirects(response, self.url_to_comments)
        # Заодно проверим статус-коды ответов.
        self.assertEqual(response.status_code, HTTPStatus.FOUND)
        # Считаем количество комментариев в системе.
        comments_count = Comment.objects.count()
        # Ожидаем ноль комментариев в системе.
        self.assertEqual(comments_count, 0)

Важно помнить, что в `django.test` каждый тест происходит в транзакции, и удаление комментария в одном тесте никак не повлияет на другие тесты в этом классе; в начале следующих тестов база данных вновь будет содержать один комментарий с исходными значениями, заданными при создании объекта комментария.

Чтобы убедиться в этом — можно добавить в начало каждого теста проверку утверждения «в БД содержится один комментарий»:


In [ ]:

# news/tests/test_logic.py
...
    def test_<любой тест в этом классе>(self):
        comments_count = Comment.objects.count()
        # В начале теста в БД всегда есть 1 комментарий, созданный в setUpTestData.
        self.assertEqual(comments_count, 1) 
        ...  # Остальные строки теста.

Эта проверка подтвердит, что состояние БД одинаковое для всех тестов. После проверки уберите этот код из тестов: оставлять подобные строки нет необходимости.

Теперь проверим, что пользователь не может удалить чужой комментарий. При попытке отправить запрос на удаление чужого комментария должна вернуться ошибка 404 — страница не найдена.


In [ ]:

# news/tests/test_logic.py
...
    def test_user_cant_delete_comment_of_another_user(self):
        # Выполняем запрос на удаление от пользователя-читателя.
        response = self.reader_client.delete(self.delete_url)
        # Проверяем, что вернулась 404 ошибка.
        self.assertEqual(response.status_code, HTTPStatus.NOT_FOUND)
        # Убедимся, что комментарий по-прежнему на месте.
        comments_count = Comment.objects.count()
        self.assertEqual(comments_count, 1) 


Аналогичным способом проверим, что редактировать комментарии может только их автор.

После редактирования комментария нужно проверить, изменилось ли его содержимое; для этого нужно сравнить поле `text` объекта комментария со значением `NEW_COMMENT_TEXT`. Однако в Python-объекте `self.comment` хранится исходное состояние комментария (то, что было до редактирования); этот объект не обновится при изменении записи в БД. 

В результате при сравнении значений `self.comment.text` и `NEW_COMMENT_TEXT` тест провалится. Необходимо обновить объект `self.comment`; для этого применим метод `refresh_from_db()`.

Во втором тесте проверим, что редактирование комментария недоступно для другого пользователя.


In [ ]:

# news/tests/test_logic.py
...
    def test_author_can_edit_comment(self):
        # Выполняем запрос на редактирование от имени автора комментария.
        response = self.author_client.post(self.edit_url, data=self.form_data)
        # Проверяем, что сработал редирект.
        self.assertRedirects(response, self.url_to_comments)
        # Обновляем объект комментария.
        self.comment.refresh_from_db()
        # Проверяем, что текст комментария соответствует обновленному.
        self.assertEqual(self.comment.text, self.NEW_COMMENT_TEXT)

    def test_user_cant_edit_comment_of_another_user(self):
        # Выполняем запрос на редактирование от имени другого пользователя.
        response = self.reader_client.post(self.edit_url, data=self.form_data)
        # Проверяем, что вернулась 404 ошибка.
        self.assertEqual(response.status_code, HTTPStatus.NOT_FOUND)
        # Обновляем объект комментария.
        self.comment.refresh_from_db()
        # Проверяем, что текст остался тем же, что и был.
        self.assertEqual(self.comment.text, self.COMMENT_TEXT)

Если при тестировании подтверждено, что авторизованный пользователь не может изменить или удалить чужой комментарий — то, как правило, в Django нет необходимости проверять, что эту операцию не сможет выполнить анонимный пользователь. В нашей ситуации такая проверка не нужна.

> Чтобы при очередных встречах с библиотекой unittest вам было проще, сохраните в закладки [шпаргалку](https://code.s3.yandex.net/Python-dev/cheatsheets/037-testirovanie-unittest-shpora/037-testirovanie-unittest-shpora.html).